# 문서 요약 (Summarization)

이번 튜토리얼에서는 **LangChain을 활용한 문서 요약 기법** 을 학습합니다.

문서 요약은 대용량 텍스트 데이터에서 핵심 정보를 추출하는 기술로, 비즈니스 환경에서 보고서 분석, 뉴스 요약, 연구 논문 정리 등에 활용됩니다.

## 학습 목표

- **다양한 요약 방식** 의 특징과 적용 시나리오 이해
- **실무 적용 가능한 요약 시스템** 구축 방법 습득  
- **효율적인 대용량 문서 처리** 기법 학습

## 목차

1. [기본 개념](#기본-개념) - 문서 요약의 핵심 과제와 해결 방안
2. [실습 데이터 소개](#실습-데이터-소개) - 소프트웨어정책연구소 보고서 활용
3. [환경 설정](#환경-설정) - 필수 라이브러리 및 API 키 설정
4. [Stuff 방식](#stuff-방식) - 전체 문서를 한 번에 요약
5. [Map-Reduce 방식](#map-reduce-방식) - 분할 요약 후 병합
6. [Map-Refine 방식](#map-refine-방식) - 점진적 개선을 통한 고품질 요약
7. [Chain of Density 방식](#chain-of-density-방식) - 반복적 개선을 통한 밀도 높은 요약
8. [Clustering-Map-Refine 방식](#clustering-map-refine-방식) - 클러스터링을 활용한 대용량 문서 처리

---

## 기본 개념

문서 요약에서 핵심적인 과제는 **긴 문서를 LLM의 컨텍스트 윈도우에 어떻게 효율적으로 전달할 것인가** 입니다.

### 컨텍스트 윈도우 제한 문제

대부분의 LLM은 **컨텍스트 윈도우 제한** 이 있습니다:

| 모델 | 컨텍스트 윈도우 | 대략적인 단어 수 |
|------|----------------|------------------|
| GPT-4 | 128K 토큰 | 96,000 단어 |
| Claude | 200K 토큰 | 150,000 단어 |

실제 업무에서는 이보다 훨씬 긴 문서들을 처리해야 할 경우가 많습니다:
- 연간 보고서 (수백 페이지)
- 연구 논문 모음
- 다수의 뉴스 기사

### 문서 요약 전략 개요

문서 요약을 위한 **4가지 검증된 전략** 을 소개합니다:

| 전략 | 처리 방식 | 적합한 문서 길이 | 품질 | 처리 속도 |
|------|-----------|------------------|------|-----------|
| **Stuff** | 전체 문서 일괄 처리 | 짧음 | 높음 | 매우 빠름 |
| **Map-Reduce** | 분할 후 병렬 처리 | 긴 | 보통 | 빠름 |
| **Map-Refine** | 분할 후 순차 개선 | 긴 | 높음 | 보통 |
| **Chain of Density** | 반복적 밀도 증가 | 중간 | 매우 높음 | 느림 |

### 각 전략의 동작 원리

#### Stuff 방식
```
전체 문서 → LLM → 요약 결과
```

#### Map-Reduce 방식  
```
문서1 → 요약1 ↘
문서2 → 요약2 → 최종 요약
문서3 → 요약3 ↗
```

#### Map-Refine 방식
```
문서1 → 요약1 → 요약1+문서2 → 개선된 요약 → ...
```

#### Chain of Density 방식
```
초기 요약 → 엔티티 추가 → 더 밀도 높은 요약 → 반복
```

### 전략 선택 가이드

| 상황 | 문서 길이 | 품질 요구도 | 처리 시간 | 추천 전략 |
|------|-----------|-------------|-----------|-----------|
| 빠른 프로토타이핑 | 짧음 | 보통 | 빠름 | **Stuff** |
| 대용량 처리 | 긴 | 보통 | 빠름 | **Map-Reduce** |
| 고품질 요약 | 긴 | 높음 | 보통 | **Map-Refine** |
| 최고품질 요구 | 중간~긴 | 최고 | 느림 | **Chain of Density** |

### 각 전략의 특징 분석

#### Stuff 방식
- **장점**: 간단하고 직관적, 빠른 처리, 비용 효율적, 맥락 보존
- **단점**: 큰 문서에는 사용 불가
- **적용 사례**: 짧은 문서, 빠른 프로토타이핑

#### Map-Reduce 방식  
- **장점**: 병렬 처리 가능, 확장성 우수
- **단점**: 문서간 연결성 손실 가능
- **적용 사례**: 독립적인 챕터들, 뉴스 모음

#### Map-Refine 방식
- **장점**: 문서 순서와 맥락 유지, 점진적 개선
- **단점**: 순차 처리로 인한 시간 소요
- **적용 사례**: 스토리가 있는 문서, 순서가 중요한 자료

#### Chain of Density 방식
- **장점**: 정보 손실 최소화, 고품질 요약
- **단점**: 여러 번의 LLM 호출로 비용 증가  
- **적용 사례**: 고품질이 중요한 보고서, 학술 자료

## 실습 데이터 소개

실습에서는 **실제 정부 기관 보고서** 를 활용하여 현실적인 문서 요약 시나리오를 경험합니다.

### 사용 데이터

**소프트웨어정책연구소(SPRi) AI Brief - 2023년 12월호**

- **저자**: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- **출처**: https://spri.kr/posts/view/23669
- **파일명**: `SPRI_AI_Brief_2023년12월호_F.pdf`

### 파일 준비

실습을 위해 다음 파일들을 `data` 폴더에 준비해주세요:

| 파일명 | 용도 | 실습 방식 |
|--------|------|----------|
| `SPRI_AI_Brief_2023년12월호_F.pdf` | 정부 보고서 | Map-Reduce, Map-Refine |
| `news.txt` | 뉴스 텍스트 | Stuff 방식 |

### 데이터 선택 이유

**현실적인 업무 시나리오** 를 반영한 데이터를 선택했습니다:

- **현실적인 길이**: 일반적인 업무 문서와 비슷한 규모  
- **구조화된 내용**: 명확한 섹션 구분으로 요약 결과 평가 용이  
- **전문 용어 포함**: 실제 비즈니스 문서의 특성 반영  
- **공개 문서**: 라이선스 문제 없이 활용 가능

## 환경 설정

문서 요약 실습을 시작하기 전에 필요한 **API 키와 추적 도구** 를 설정합니다.

### 필요한 설정

1. **OpenRouter API Key**: GPT 모델 사용을 위해 필요
2. **LangSmith 추적**: 실행 과정을 모니터링하고 디버깅을 위해 사용

이 설정들이 완료되면 모든 LangChain 체인의 실행 과정을 **LangSmith 대시보드** 에서 실시간으로 확인할 수 있습니다.

# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

## Stuff 방식

**Stuff 방식** 은 문서 요약의 가장 기본적이고 직관적인 접근 방법입니다.

### 동작 원리

**"Stuff"** 라는 이름은 **모든 문서를 프롬프트 안에 집어넣는다** 는 의미에서 유래했습니다.

```
전체 문서 → 단일 프롬프트 → LLM → 요약 결과
```

### 기술적 특징

| 항목 | 설명 |
|------|------|
| **처리 방식** | 전체 문서를 한 번에 LLM에 전달 |
| **API 호출 수** | 1회 (최소) |
| **병렬 처리** | 불가능 |
| **맥락 보존** | 완벽 (전체 문서 맥락 유지) |

### 장점과 제약 사항

#### 장점
- **단순성**: 가장 이해하기 쉬운 방식
- **속도**: 단 한 번의 LLM 호출로 완료
- **비용 효율성**: API 호출 최소화
- **맥락 보존**: 전체 문서의 맥락을 완벽히 유지

#### 제약 사항  
- **크기 제한**: LLM의 컨텍스트 윈도우 크기에 의존
- **확장성 부족**: 대용량 문서에는 적용 불가

### 적용 기준

#### 높은 적합성
- **짧은 문서**: 뉴스 기사, 블로그 포스트
- **빠른 프로토타이핑**: 개념 검증용 개발  
- **비용 절약**: 예산이 제한적인 프로젝트
- **단순 요약**: 복잡한 처리가 불필요한 경우

#### 낮은 적합성
- **대용량 문서**: 컨텍스트 윈도우를 초과하는 문서
- **복잡한 구조**: 여러 섹션으로 구성된 긴 보고서
- **병렬 처리 필요**: 다수의 문서를 동시에 처리해야 하는 경우

### 실무 활용 통계

실제로 **업무 문서의 80% 이상** 은 Stuff 방식으로도 충분히 처리 가능합니다:

| 문서 유형 | 평균 길이 | Stuff 방식 적용 가능성 |
|----------|-----------|----------------------|
| 뉴스 기사 | 1-3 페이지 | 매우 높음 |
| 블로그 포스트 | 2-5 페이지 | 높음 |
| 업무 보고서 | 5-10 페이지 | 보통 |
| 연구 논문 | 10-30 페이지 | 낮음 |

### 데이터 로드

실습을 위해 **뉴스 텍스트 파일** 을 로드합니다. Stuff 방식은 상대적으로 짧은 문서에 적합하므로, PDF보다는 텍스트 파일을 사용합니다.

In [ ]:
from langchain_community.document_loaders import TextLoader

# 뉴스데이터 로드
loader = TextLoader("data/news.txt")
docs = loader.load()

# 문서의 총 글자수 출력
print(f"총 글자수: {len(docs[0].page_content)}")
print("\n========= 앞부분 미리보기 =========\n")

# 문서의 앞부분 500자 미리보기
print(docs[0].page_content[:500])

### 프롬프트 설정

문서 요약을 위한 **전용 프롬프트** 를 로드합니다. 이 프롬프트는 한국어 요약에 최적화되어 있으며, 구조화된 형태로 가독성을 높입니다.

In [ ]:
from langchain import hub

# 한국어 요약 전용 프롬프트 로드
prompt = hub.pull("teddynote/summary-stuff-documents-korean")

# 프롬프트 구조 출력 및 확인
prompt.pretty_print()

In [ ]:
# from langchain_core.prompts import PromptTemplate

# prompt = PromptTemplate.from_template(
#     """Please summarize the sentence according to the following REQUEST.
# REQUEST:
# 1. Summarize the main points in bullet points in KOREAN.
# 2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
# 3. Use various emojis to make the summary more interesting.
# 4. Translate the summary into KOREAN if it is written in ENGLISH.
# 5. DO NOT translate any technical terms.
# 6. DO NOT include any unnecessary information.

# CONTEXT:
# {context}

# SUMMARY:"
# """
# )

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_teddynote.callbacks import StreamingCallback

# ChatOpenAI 모델 설정 (OpenRouter 사용)
llm = ChatOpenAI(
    model="openai/gpt-4.1",  # OpenRouter에서 지원하는 모델명
    api_key=os.getenv("OPENROUTER_API_KEY"),  # OpenRouter API 키
    base_url=os.getenv("OPENROUTER_BASE_URL"),  # OpenRouter 기본 URL
    streaming=True,  # 스트리밍 출력 활성화
    temperature=0,  # 출력의 일관성을 위해 temperature를 0으로 설정
    callbacks=[StreamingCallback()],  # 실시간 출력을 위한 콜백
)

# Stuff 방식의 문서 요약 체인 생성
stuff_chain = create_stuff_documents_chain(llm, prompt)

# 문서를 입력으로 요약 실행
answer = stuff_chain.invoke({"context": docs})

## Map-Reduce 방식

**Map-Reduce 방식** 은 대용량 문서를 효율적으로 처리하기 위한 분할 정복 전략입니다.

### 동작 원리

Map-Reduce는 **2단계 처리 과정** 으로 구성됩니다:

#### 1단계: Map (분할 처리)
```
긴 문서 → [청크1, 청크2, 청크3] → [요약1, 요약2, 요약3]
```

#### 2단계: Reduce (결과 통합)  
```
[요약1, 요약2, 요약3] → LLM → 최종 통합 요약
```

### 기술적 특징

| 항목 | 설명 |
|------|------|
| **처리 방식** | 분할 후 병렬 처리 |
| **API 호출 수** | N+1회 (N=청크 수) |
| **병렬 처리** | 가능 (Map 단계) |
| **맥락 보존** | 부분적 (청크 간 연결 제한) |

### 장점과 제약 사항

#### 장점
- **병렬 처리**: 각 청크를 동시에 처리하여 시간 단축
- **확장성**: 문서 길이에 관계없이 처리 가능
- **비용 효율성**: 각 청크별로 독립적 처리

#### 제약 사항
- **맥락 연결**: 청크 간의 맥락 연결이 약해질 수 있음
- **순서 의존성**: 순서가 중요한 문서에서는 정보 손실 가능

### 성능 비교

| 특성 | Map-Reduce | Stuff | Map-Refine |
|------|------------|-------|------------|
| **처리 속도** | 빠름 | 매우 빠름 | 느림 |
| **맥락 보존** | 보통 | 높음 | 높음 |
| **확장성** | 높음 | 낮음 | 높음 |
| **비용** | 보통 | 낮음 | 높음 |
| **병렬 처리** | 가능 | 불가능 | 불가능 |

### 최적 활용 시나리오

#### 높은 적합성
| 문서 유형 | 적용도 | 이유 |
|----------|--------|------|
| 독립적인 챕터들 | 매우 높음 | 각 섹션이 상대적으로 독립적 |
| 뉴스 모음 | 높음 | 여러 기사를 하나로 요약 |
| 월간 보고서 모음 | 높음 | 각 월별 내용이 독립적 |
| 제품 리뷰 모음 | 높음 | 각 리뷰가 독립적 |

#### 낮은 적합성
| 문서 유형 | 적용도 | 이유 |
|----------|--------|------|
| 연속된 스토리 | 낮음 | 맥락 연결 중요 |
| 시계열 분석 | 낮음 | 순서와 연관성 중요 |
| 논증형 문서 | 낮음 | 논리적 흐름 보존 필요 |

### 실무 구현 고려사항

#### Map 단계 최적화
- **청크 크기 조정**: 너무 작으면 맥락 손실, 너무 크면 처리 지연
- **중복 구간 설정**: 청크 간 연결성 확보
- **핵심 정보 추출**: 요약보다는 핵심 내용 추출 권장

#### Reduce 단계 최적화
- **정보 통합**: 중복 제거 및 일관성 확보
- **구조화**: 논리적 흐름으로 재구성
- **품질 검증**: 전체 맥락에서의 일관성 확인

### 데이터 로드

이번에는 **정부 연구기관의 AI 보고서** 를 활용합니다. Stuff 방식과 달리 **더 긴 문서** 를 처리하므로 PDF 파일을 사용합니다.

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

# PDF 파일 로드
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2025_08.pdf")
docs = loader.load()

# 실습을 위해 문서의 일부만 사용 (3-8페이지)
docs = docs[3:8]

# 총 페이지 수 출력
print(f"총 페이지수: {len(docs)}")

### Map 단계 - 분할 처리

**Map 단계** 에서는 각 문서 청크에 대해 **핵심 내용을 추출** 합니다.

#### 전략적 선택

일반적으로 Map 단계에서는 **요약** 을 생성하지만, 여기서는 **핵심 내용 추출** 방식을 사용합니다:

**핵심 내용 추출의 장점**
- **정보 손실 최소화**: 중요한 세부사항 보존
- **Reduce 단계 효율성**: 더 나은 최종 요약 품질
- **유연성**: 다양한 문서 타입에 적응 가능

**요약 vs 핵심 내용 추출**
- **요약**: 각 청크를 짧게 요약하여 정보 압축
- **핵심 내용 추출**: 중요한 정보만 선별하여 정보 보존

**실무 팁**: 최종 품질이 중요하다면 핵심 내용 추출을, 빠른 처리가 필요하다면 요약을 선택하세요.

In [ ]:
import os
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델 설정 (Map 단계용, OpenRouter 사용)
llm = ChatOpenAI(
    model="openai/gpt-4.1",  # OpenRouter에서 지원하는 모델명
    api_key=os.getenv("OPENROUTER_API_KEY"),  # OpenRouter API 키
    base_url=os.getenv("OPENROUTER_BASE_URL"),  # OpenRouter 기본 URL
    temperature=0,  # 일관된 추출을 위해 temperature 0 설정
)

# Map 단계용 프롬프트 다운로드
map_prompt = hub.pull("teddynote/map-prompt")

# 프롬프트 구조 출력 및 확인
map_prompt.pretty_print()

### Map Chain 생성

Map 프롬프트와 LLM을 연결하여 **핵심 내용 추출 체인** 을 만듭니다.

In [ ]:
# Map 체인 생성: 프롬프트 → LLM → 문자열 출력파서
map_chain = map_prompt | llm | StrOutputParser()

### 병렬 처리 실행

**batch() 메서드** 를 사용하여 **모든 문서를 동시에 처리** 합니다. 이는 Map-Reduce의 핵심 장점인 병렬 처리를 활용하는 부분입니다.

In [ ]:
# 모든 문서에 대해 병렬로 핵심 내용 추출 실행
doc_summaries = map_chain.batch(docs)

In [ ]:
# 추출된 핵심 내용의 개수 확인
len(doc_summaries)

In [ ]:
# 첫 번째 문서의 핵심 내용 확인
print(doc_summaries[0])

### Reduce 단계 - 결과 통합

**Reduce 단계** 에서는 각 문서에서 추출한 **핵심 내용들을 하나의 완전한 요약** 으로 통합합니다.

#### Reduce 단계의 역할

1. **정보 종합**: 분산된 핵심 내용들을 연결
2. **중복 제거**: 반복되는 내용을 정리
3. **구조화**: 논리적으로 일관된 요약 생성
4. **최종 다듬기**: 읽기 쉬운 형태로 가공

In [ ]:
# Reduce 단계용 프롬프트 다운로드
reduce_prompt = hub.pull("teddynote/reduce-prompt")

# 프롬프트 구조 출력 및 확인
reduce_prompt.pretty_print()

### Reduce Chain 생성

여러 핵심 내용들을 **최종 요약으로 통합** 하는 Reduce 체인을 생성합니다.

In [ ]:
# Reduce 체인 생성: 프롬프트 → LLM → 문자열 출력파서
reduce_chain = reduce_prompt | llm | StrOutputParser()

### 스트리밍 결과 확인

Reduce Chain을 실행하여 **실시간으로 최종 요약 결과** 를 확인해봅시다.

In [ ]:
from langchain_teddynote.messages import stream_response

# Reduce 체인을 스트리밍 모드로 실행
answer = reduce_chain.stream(
    {
        "doc_summaries": "\n".join(doc_summaries),
        "language": "Korean",
    }  # 추출된 핵심내용들을 하나로 합쳐서 입력
)

# 실시간 스트리밍 출력
stream_response(answer)

In [ ]:
import os
from langchain_core.runnables import chain


@chain
def map_reduce_chain(docs):
    """Map-Reduce 전체 과정을 하나의 체인으로 통합한 함수"""

    # Map 단계용 LLM 설정 (OpenRouter 사용)
    map_llm = ChatOpenAI(
        model="openai/gpt-4.1",  # OpenRouter에서 지원하는 모델명
        api_key=os.getenv("OPENROUTER_API_KEY"),  # OpenRouter API 키
        base_url=os.getenv("OPENROUTER_BASE_URL"),  # OpenRouter 기본 URL
        temperature=0,
    )

    # Map 프롬프트 다운로드
    map_prompt = hub.pull("teddynote/map-prompt")

    # Map 체인 생성
    map_chain = map_prompt | map_llm | StrOutputParser()

    # 첫 번째 단계: 모든 문서에서 핵심 내용 추출 (Map)
    doc_summaries = map_chain.batch(docs)

    # Reduce 프롬프트 다운로드
    reduce_prompt = hub.pull("teddynote/reduce-prompt")

    # Reduce 단계용 LLM 설정 (스트리밍 포함, OpenRouter 사용)
    reduce_llm = ChatOpenAI(
        model="openai/gpt-4.1",  # OpenRouter에서 지원하는 모델명
        api_key=os.getenv("OPENROUTER_API_KEY"),  # OpenRouter API 키
        base_url=os.getenv("OPENROUTER_BASE_URL"),  # OpenRouter 기본 URL
        temperature=0,
        callbacks=[StreamingCallback()],  # 실시간 출력
        streaming=True,
    )

    # Reduce 체인 생성
    reduce_chain = reduce_prompt | reduce_llm | StrOutputParser()

    # 두 번째 단계: 핵심 내용들을 최종 요약으로 통합 (Reduce)
    return reduce_chain.invoke(
        {"doc_summaries": "\n".join(doc_summaries), "language": "Korean"}
    )

In [ ]:
# Map-Reduce 통합 체인 실행 (전체 과정을 한번에)
answer = map_reduce_chain.invoke(docs)

## Map-Refine 방식

**Map-Refine 방식** 은 점진적 개선을 통해 고품질 요약을 생성하는 전략입니다.

### 동작 원리

Map-Refine은 **순차적 개선 과정** 으로 진행됩니다:

#### Map 단계 - 초기 요약 생성
```
각 문서 청크 → LLM → 개별 요약들
```

#### Refine 단계 - 점진적 개선
```
요약1 + 문서2 → LLM → 개선된 요약
개선된 요약 + 문서3 → LLM → 더 개선된 요약
...
```

### 핵심 특징

#### 장점
- **맥락 보존**: 문서 순서와 연결성 유지
- **점진적 개선**: 각 단계에서 품질 향상
- **정보 누적**: 이전 정보를 잃지 않고 새 정보 추가

#### 주의사항
- **순차 처리**: 병렬 처리 불가로 시간 소요
- **비용 증가**: Map-Reduce보다 많은 LLM 호출
- **순서 의존성**: 문서 순서가 결과에 영향

### Map-Reduce vs Map-Refine 비교

| 특성 | Map-Reduce | Map-Refine |
|------|------------|------------|
| **처리 방식** | 병렬 처리 | 순차 처리 |
| **맥락 보존** | 보통 | 우수 |
| **처리 속도** | 빠름 | 느림 |
| **품질** | 좋음 | 더 좋음 |
| **비용** | 저렴 | 비쌈 |
| **적용 상황** | 독립적 문서 | 연속적 문서 |

### 최적 활용 시나리오

#### 높은 적용도
- **연속된 스토리**: 소설, 연대기, 일지 등
- **시계열 데이터**: 월별/분기별 보고서의 추이 분석
- **상세한 분석**: 품질이 속도보다 중요한 경우

#### 낮은 적용도  
- **독립적 섹션**: 각 챕터가 독립적인 문서
- **긴급한 처리**: 빠른 결과가 필요한 경우
- **제한된 예산**: 비용 절약이 중요한 상황

### Map 단계 - 개별 요약 생성

**Map-Refine의 Map 단계** 에서는 각 문서 청크에 대해 **개별 요약** 을 생성합니다.

#### Map-Reduce와의 차이점

- **Map-Reduce**: 핵심 내용 추출 (키워드, 중요 정보)
- **Map-Refine**: 완성된 요약 생성 (읽을 수 있는 요약문)

Map-Refine에서는 각 청크마다 **완전한 요약문** 을 만드는 것이 중요합니다.

In [ ]:
import os
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Map 단계용 LLM 생성 (OpenRouter 사용)
map_llm = ChatOpenAI(
    model="openai/gpt-4.1",  # OpenRouter에서 지원하는 모델명
    api_key=os.getenv("OPENROUTER_API_KEY"),  # OpenRouter API 키
    base_url=os.getenv("OPENROUTER_BASE_URL"),  # OpenRouter 기본 URL
    temperature=0,  # 일관된 요약을 위해 temperature 0 설정
)

# Map-Refine 전용 요약 프롬프트 다운로드
map_summary = hub.pull("teddynote/map-summary-prompt")

# 프롬프트 구조 출력 및 확인
map_summary.pretty_print()

### Map Chain 생성

요약용 프롬프트와 LLM을 연결하여 **개별 문서 요약 체인** 을 만듭니다.

In [ ]:
# Map 체인 생성: 프롬프트 → LLM → 문자열 출력파서
map_chain = map_summary | map_llm | StrOutputParser()

### 단일 문서 요약 테스트

첫 번째 문서에 대한 요약을 생성하여 **Map 단계의 출력 형태** 를 확인해봅시다.

In [ ]:
# 첫 번째 문서의 개별 요약 생성 및 출력
print(map_chain.invoke({"documents": docs[0], "language": "Korean"}))

In [ ]:
# 배치 처리를 위해 모든 문서를 동일한 형태로 변환
input_doc = [{"documents": doc, "language": "Korean"} for doc in docs]

In [ ]:
# 변환된 입력 형태 확인
input_doc

In [ ]:
# 모든 문서에 대한 개별 요약 생성 (병렬 처리)
print(map_chain.batch(input_doc))

### Refine 단계 - 점진적 개선

**Refine 단계** 에서는 **이전 요약 + 새 요약** 을 합쳐서 **더 나은 요약** 으로 점진적으로 개선합니다.

#### Refine 과정

1. **첫 번째**: 요약1을 기준으로 시작
2. **두 번째**: 요약1 + 요약2 → 개선된 요약A  
3. **세 번째**: 개선된 요약A + 요약3 → 개선된 요약B
4. **반복**: 모든 요약이 통합될 때까지 계속

이 과정을 통해 **정보의 누적과 정제** 가 동시에 일어납니다.

In [ ]:
# Refine 단계용 프롬프트 다운로드
refine_prompt = hub.pull("teddynote/refine-prompt")

# 프롬프트 구조 출력 및 확인
refine_prompt.pretty_print()

In [ ]:
import os

# Refine 단계용 LLM 생성 (OpenRouter 사용)
refine_llm = ChatOpenAI(
    model="openai/gpt-4.1",  # OpenRouter에서 지원하는 모델명
    api_key=os.getenv("OPENROUTER_API_KEY"),  # OpenRouter API 키
    base_url=os.getenv("OPENROUTER_BASE_URL"),  # OpenRouter 기본 URL
    temperature=0,  # 일관된 개선을 위해 temperature 0 설정
)

# Refine 체인 생성: 프롬프트 → LLM → 문자열 출력파서
refine_chain = refine_prompt | refine_llm | StrOutputParser()

### Map-Refine 통합 체인

지금까지의 **Map과 Refine 과정을 하나의 체인** 으로 통합합니다.

#### 전체 과정

1. **Map**: 각 문서의 개별 요약 생성
2. **Initialize**: 첫 번째 요약을 기준으로 설정  
3. **Refine Loop**: 나머지 요약들을 순차적으로 통합
4. **Final**: 최종 통합 요약 완성

In [ ]:
import os
from langchain_core.runnables import chain


@chain
def map_refine_chain(docs):
    """Map-Refine 전체 과정을 하나의 체인으로 통합한 함수"""

    # Map 단계: 각 문서에 대한 개별 요약 생성
    map_summary = hub.pull("teddynote/map-summary-prompt")

    map_chain = (
        map_summary
        | ChatOpenAI(
            model="openai/gpt-4.1",  # OpenRouter에서 지원하는 모델명
            api_key=os.getenv("OPENROUTER_API_KEY"),  # OpenRouter API 키
            base_url=os.getenv("OPENROUTER_BASE_URL"),  # OpenRouter 기본 URL
            temperature=0,
        )
        | StrOutputParser()
    )

    # 모든 문서를 배치 처리용 형태로 변환
    input_doc = [{"documents": doc.page_content, "language": "Korean"} for doc in docs]

    # 첫 번째 단계: 모든 문서에 대한 개별 요약 생성 (Map)
    doc_summaries = map_chain.batch(input_doc)

    # Refine 단계 설정
    refine_prompt = hub.pull("teddynote/refine-prompt")

    refine_llm = ChatOpenAI(
        model="openai/gpt-4.1",  # OpenRouter에서 지원하는 모델명
        api_key=os.getenv("OPENROUTER_API_KEY"),  # OpenRouter API 키
        base_url=os.getenv("OPENROUTER_BASE_URL"),  # OpenRouter 기본 URL
        temperature=0,
        callbacks=[StreamingCallback()],  # 실시간 출력
        streaming=True,
    )

    refine_chain = refine_prompt | refine_llm | StrOutputParser()

    # 두 번째 단계: 순차적 개선 과정 (Refine)
    previous_summary = doc_summaries[0]  # 첫 번째 요약으로 시작

    # 나머지 요약들을 순차적으로 통합
    for current_summary in doc_summaries[1:]:

        previous_summary = refine_chain.invoke(
            {
                "previous_summary": previous_summary,  # 이전까지의 통합된 요약
                "current_summary": current_summary,  # 새로 추가할 요약
                "language": "Korean",
            }
        )
        print("\n\n-----------------\n\n")  # 각 단계 구분을 위한 출력

    return previous_summary

In [ ]:
# Map-Refine 통합 체인 실행 (점진적 개선 과정을 실시간으로 확인)
refined_summary = map_refine_chain.invoke(docs)

## Chain of Density 방식

**Chain of Density (CoD)** 는 MIT 연구팀이 개발한 혁신적인 요약 기법으로, 반복적으로 요약의 **정보 밀도를 높여가는** 방식입니다.

### 연구 배경

#### 학술적 기반
- **논문**: [From Sparse to Dense: GPT-4 Summarization with Chain of Density Prompting](https://arxiv.org/pdf/2309.04269)
- **연구 기관**: MIT
- **핵심 발견**: 한 번에 완벽한 요약보다는 점진적 개선이 더 효과적

#### 개발 목적
**정보량과 가독성의 최적 균형점** 을 찾는 것이 주요 목표입니다.

### 동작 원리

CoD는 **4단계의 점진적 개선** 으로 진행됩니다:

| 단계 | 과정 | 목표 |
|------|------|------|
| **1단계** | 초기 요약 생성 | 간단하고 이해하기 쉬운 요약 |
| **2단계** | 중요 엔티티 식별 | 누락된 핵심 정보 파악 |
| **3단계** | 밀도 증가 | 길이는 유지하며 정보량 증대 |
| **4단계** | 반복 개선 | 최적 밀도까지 반복 |

### 기술적 특징

| 항목 | 설명 |
|------|------|
| **처리 방식** | 반복적 밀도 증가 |
| **API 호출 수** | 3-5회 (반복 횟수에 따라) |
| **병렬 처리** | 불가능 (순차적 개선) |
| **정보 보존** | 매우 높음 (단계적 보완) |

### 핵심 매개변수

#### 주요 설정값
| 매개변수 | 기본값 | 설명 | 조정 가이드 |
|----------|--------|------|-------------|
| **max_words** | 80 | 요약 최대 단어 수 | 문서 복잡도에 따라 60-120 |
| **entity_range** | 1-3 | 한 번에 추가할 엔티티 수 | 신중함 vs 속도의 균형 |
| **iterations** | 3-5 | 반복 횟수 | 품질 요구도에 따라 조정 |
| **content_category** | Article | 콘텐츠 유형 | 문서 특성에 맞게 설정 |

#### 매개변수 조합 전략
| 문서 복잡도 | max_words | iterations | entity_range | 적용 사례 |
|-------------|-----------|------------|--------------|-----------|
| **낮음** | 60 | 3 | 1-2 | 뉴스, 간단한 글 |
| **보통** | 80 | 3-4 | 1-3 | 일반 보고서, 기사 |
| **높음** | 100 | 4 | 1-3 | 연구 논문, 전문 자료 |
| **매우 높음** | 120 | 4-5 | 1-4 | 복잡한 학술 자료 |

### 성능 및 효과

#### 검증된 효과
- **정보 밀도**: 일반 GPT-4 요약보다 높은 정보 밀도 달성
- **품질**: 인간이 작성한 요약과 비슷한 수준
- **Lead bias 감소**: 문서 앞부분 편향성 해결

#### 다른 방식과의 비교
| 특성 | Chain of Density | Map-Reduce | Map-Refine | Stuff |
|------|------------------|------------|------------|-------|
| **정보 밀도** | 최고 | 보통 | 높음 | 높음 |
| **처리 시간** | 느림 | 빠름 | 보통 | 매우 빠름 |
| **비용** | 높음 | 보통 | 높음 | 낮음 |
| **품질 일관성** | 매우 높음 | 보통 | 높음 | 높음 |

### 최적 활용 시나리오

#### 높은 적합성
- **중요 보고서**: 정보 손실을 최소화해야 하는 문서
- **뉴스 요약**: 핵심 사실들을 놓치지 않고 전달
- **연구 논문**: 복잡한 내용을 정확히 압축
- **비즈니스 문서**: 의사결정에 필요한 모든 정보 보존

#### 제한 사항
- **처리 시간**: 다른 방식 대비 시간 소요
- **비용**: 여러 번의 API 호출로 비용 증가
- **문서 길이**: 매우 긴 문서에는 적합하지 않음

### 실무 적용 팁

#### 최적화 전략
- **문서 전처리**: 불필요한 내용 제거 후 적용
- **배치 처리**: 유사한 문서들을 묶어서 처리
- **품질 모니터링**: 각 단계별 개선 효과 추적
- **비용 관리**: 중요도에 따른 선택적 적용

### 프롬프트 확인

실제 CoD에서 사용하는 프롬프트는 [LangSmith Hub](https://smith.langchain.com/prompts/chain-of-density-prompt/4582aae0?organizationId=8c9eeb3c-2665-5405-bc50-0767fdf4ca8f)에서 확인할 수 있습니다.

### 입력 파라미터 상세 설명

CoD 체인의 성능을 좌우하는 핵심 매개변수들을 살펴봅시다:

#### **content_category** (콘텐츠 유형)
- **설명**: 요약할 콘텐츠의 종류를 명시
- **기본값**: `"Article"`
- **예시**: "Article", "Research Paper", "News", "Report"
- **효과**: 콘텐츠 유형에 맞는 요약 스타일 적용

#### **content** (요약할 내용)
- **설명**: 실제 요약하고자 하는 텍스트 전문
- **형태**: 문자열 (string)
- **길이**: 제한 없음 (단, LLM 컨텍스트 윈도우 고려)

#### **entity_range** (엔티티 개수 범위) 
- **설명**: 각 반복에서 추가할 엔티티 수
- **기본값**: `"1-3"`  
- **조정 가이드**:
  - `"1-2"`: 신중한 개선 (더 정확, 더 느림)
  - `"1-3"`: 균형잡힌 개선 (추천)  
  - `"2-4"`: 빠른 개선 (더 빠름, 품질 저하 가능)

#### **max_words** (최대 단어 수)
- **설명**: 생성될 요약의 최대 길이 제한
- **기본값**: `80`
- **조정 가이드**:
  - **50-80**: 간결한 요약 (빠른 파악용)
  - **80-120**: 표준 요약 (일반적 용도)
  - **120-150**: 상세 요약 (전문적 분석용)

#### **iterations** (반복 횟수)
- **설명**: 밀도 개선을 수행할 총 횟수
- **기본값**: `3`  
- **조정 가이드**:
  - **2-3회**: 간단한 문서, 빠른 처리
  - **3-4회**: 일반적인 문서 (추천)
  - **4-5회**: 복잡한 문서, 최고 품질 필요
  - **5회 이상**: 일반적으로 비추천 (수익 감소)

### 매개변수 조합 전략

| 문서 복잡도 | max_words | iterations | entity_range | 용도 |
|-------------|-----------|------------|--------------|------|
| **낮음** | 60 | 3 | 1-2 | 뉴스, 간단한 글 |
| **보통** | 80 | 3 | 1-3 | 일반 보고서, 기사 |  
| **높음** | 100 | 4 | 1-3 | 연구 논문, 전문 자료 |
| **매우 높음** | 120 | 4-5 | 1-4 | 복잡한 학술 자료 |

### CoD 체인 구성

이제 **Chain of Density 체인** 을 실제로 구현해봅시다.

#### 체인 설계

1. **기본 체인**: 전체 과정(중간 결과 포함) 출력  
2. **최종 요약 체인**: 최종 결과만 깔끔하게 추출

#### 핵심 아이디어

- **JSON 출력**: 각 반복 단계별 결과를 구조화된 형태로 저장
- **점진적 개선**: 이전 결과를 바탕으로 더 나은 요약 생성  
- **실시간 모니터링**: 각 단계별 개선 과정을 관찰 가능

In [ ]:
# Chain of Density 전용 프롬프트 다운로드
cod_prompt = hub.pull("teddynote/chain-of-density-prompt")

# 프롬프트 구조 출력 및 확인
cod_prompt.pretty_print()

In [ ]:
import os
import textwrap
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import SimpleJsonOutputParser

# CoD 체인의 모든 입력 파라미터에 대한 기본값 설정
cod_chain_inputs = {
    "content": lambda d: d.get("content"),  # 요약할 내용 (필수)
    "content_category": lambda d: d.get("content_category", "Article"),  # 콘텐츠 유형
    "entity_range": lambda d: d.get("entity_range", "1-3"),  # 엔티티 개수 범위
    "max_words": lambda d: int(d.get("max_words", 80)),  # 최대 단어 수
    "iterations": lambda d: int(d.get("iterations", 5)),  # 반복 횟수
}

# Chain of Density 프롬프트 다운로드
cod_prompt = hub.pull("teddynote/chain-of-density-prompt")

# 기본 CoD 체인 생성 (전체 과정 출력, OpenRouter 사용)
cod_chain = (
    cod_chain_inputs
    | cod_prompt
    | ChatOpenAI(
        model="openai/gpt-4.1",  # OpenRouter에서 지원하는 모델명
        api_key=os.getenv("OPENROUTER_API_KEY"),  # OpenRouter API 키
        base_url=os.getenv("OPENROUTER_BASE_URL"),  # OpenRouter 기본 URL
        temperature=0,
    )
    | SimpleJsonOutputParser()  # JSON 형태로 파싱
)

# 최종 요약만 추출하는 체인 생성
cod_final_summary_chain = cod_chain | (
    lambda output: output[-1].get(
        "denser_summary", '오류: 마지막 딕셔너리에 "denser_summary" 키가 없습니다'
    )
)

### 요약 대상 데이터 확인

CoD 실습을 위해 **두 번째 페이지** 의 내용을 사용하겠습니다. 이는 적절한 길이와 복잡도를 가진 텍스트입니다.

In [ ]:
# 두 번째 문서(페이지)의 내용을 CoD 입력으로 사용
content = docs[1].page_content
print(content)

### 실시간 CoD 과정 관찰

CoD의 **점진적 개선 과정** 을 실시간으로 관찰할 수 있습니다.

#### 스트리밍 동작 원리

- **JSON 스트리밍**: 각 단계별 결과가 실시간으로 업데이트
- **캐리지 리턴** (`\r`): 같은 줄에 계속 덮어쓰며 진행 상황 표시  
- **점진적 추가**: 새로운 반복이 완료될 때마다 결과 추가

#### 출력 형태

각 반복마다 다음 정보가 표시됩니다:
- **missing_entities**: 새로 추가된 엔티티들
- **denser_summary**: 개선된 요약문

In [ ]:
# 결과를 저장할 빈 리스트 초기화
results: list[dict[str, str]] = []

# CoD 체인을 스트리밍 모드로 실행하여 점진적 개선 과정 실시간 관찰
for partial_json in cod_chain.stream(
    {"content": content, "content_category": "Article"}  # 입력 콘텐츠와 유형 지정
):
    # 각 반복마다 results를 업데이트 (점진적으로 더 많은 요약이 추가됨)
    results = partial_json

    # 현재까지의 결과를 같은 줄에 출력 (캐리지 리턴으로 덮어씀)
    print(results, end="\r", flush=True)

# 스트리밍 완료 후 줄바꿈
print("\n")

# 생성된 총 요약 단계 수 계산
total_summaries = len(results)

# 각 요약 단계별로 상세 분석 및 출력
i = 1
for cod in results:
    # 새로 추가된 엔티티들을 쉼표로 구분하여 정리
    added_entities = ", ".join(
        [
            ent.strip()
            for ent in cod.get(
                "missing_entities", 'ERR: "missing_entities" key not found'
            ).split(";")
        ]
    )

    # 해당 단계의 개선된 요약 추출
    summary = cod.get("denser_summary", 'ERR: missing key "denser_summary"')

    # 단계별 헤더 출력 (몇 번째 단계인지, 추가된 엔티티 표시)
    print(
        f"### CoD Summary {i}/{total_summaries}, 추가된 엔티티(entity): {added_entities}"
        + "\n"
    )

    # 요약 내용을 80자 너비로 줄바꿈하여 가독성 있게 출력
    print(textwrap.fill(summary, width=80) + "\n")
    i += 1

# 최종 결과 구분선과 함께 출력
print("\n============== [최종 요약] =================\n")
print(summary)

In [ ]:
# 최종 요약 결과 재확인
print(summary)

## Clustering-Map-Refine 방식

**Clustering-Map-Refine** 은 대용량 문서 처리의 혁신적 접근법으로, 비용 효율성과 품질을 동시에 달성하는 지능형 요약 전략입니다.

### 개발 배경 및 철학

이 방법은 **gkamradt** 가 제안한 창의적 솔루션으로, 다음과 같은 문제의식에서 출발했습니다:

#### 기존 방식의 문제점
- **Map-Reduce**: 빠르지만 맥락 손실 가능
- **Map-Refine**: 고품질이지만 시간과 비용이 많이 소요
- **CoD**: 최고 품질이지만 매우 비쌈

#### 해결 아이디어
> **"모든 문서를 처리할 필요가 있을까? 대표적인 문서들만 골라서 고품질로 처리하면 어떨까?"**

### 동작 원리

Clustering-Map-Refine은 **3단계 지능형 처리** 로 진행됩니다:

#### 1. 문서 분할 & 임베딩
```
대용량 문서 → [청크1, 청크2, ..., 청크N] → [벡터1, 벡터2, ..., 벡터N]
```

#### 2. 클러스터링  
```
N개 벡터 → KMeans → K개 클러스터 → 각 클러스터의 중심점 문서 선택
```

#### 3. 선별된 문서로 Map-Refine
```
대표 문서들 → Map-Refine → 고품질 최종 요약
```

### 핵심 아이디어

#### 대표성 기반 선택
- **클러스터 중심**: 각 클러스터에서 가장 **대표적인 문서** 선택
- **의미적 유사성**: 비슷한 내용끼리 그룹핑하여 **중복 최소화**
- **효율적 샘플링**: 전체 문서의 **핵심 정보만 추출**

#### 처리 효율성
- **문서 수 감소**: 79개 → 10개 (약 87% 감소)  
- **비용 절감**: LLM 호출 횟수 대폭 감소
- **시간 단축**: 처리 시간 현저히 단축

### 성능 비교

| 방식 | 문서 수 | 처리 시간 | 비용 | 품질 | 적용 상황 |
|------|---------|-----------|------|------|-----------| 
| **전체 Map-Refine** | 79개 전체 | 매우 느림 | 매우 비쌈 | 최고 | 연구용 |
| **Clustering-Map-Refine** | 10개 선별 | 보통 | 합리적 | 우수 | **실무용** |
| **Map-Reduce** | 79개 전체 | 빠름 | 저렴 | 보통 | 빠른 처리용 |

### 최적 활용 시나리오

- **대용량 보고서**: 수십, 수백 페이지의 종합 보고서
- **뉴스 아카이브**: 장기간 축적된 뉴스 모음 요약
- **연구 논문 컬렉션**: 특정 주제의 다수 논문 통합 분석
- **기업 문서**: 월별/분기별 보고서 통합

### 기술적 구현 요소

1. **임베딩 모델**: 문서의 의미를 벡터로 변환
2. **클러스터링**: KMeans 알고리즘으로 유사 문서 그룹핑  
3. **중심점 선택**: 각 클러스터에서 가장 대표적인 문서 식별
4. **Map-Refine 적용**: 선별된 문서들로 고품질 요약 생성

### 실무 적용 팁

#### 최적화 전략
- **클러스터 수 조정**: 문서 복잡도에 따라 5-15개 사이에서 조정  
- **임베딩 모델 선택**: 도메인에 특화된 모델 사용 시 성능 향상  
- **품질 검증**: 선택된 대표 문서들이 전체를 잘 대표하는지 확인  
- **하이브리드 접근**: 중요도가 높은 문서는 별도로 추가 처리

#### 성공 사례
원 저자 gkamradt의 실험 결과:
- **비용**: 기존 대비 80% 절감
- **품질**: 전체 처리 대비 90% 수준 유지  
- **만족도**: 실무진들의 높은 활용도

### 참고 자료
- [gkamradt의 원본 튜토리얼](https://github.com/gkamradt/langchain-tutorials/blob/main/data_generation/5%20Levels%20Of%20Summarization%20-%20Novice%20To%20Expert.ipynb)

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

# 대용량 PDF 문서 로드 (전체 페이지)
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2025_08.pdf")
docs = loader.load()

# 총 페이지 수 확인
len(docs)

### 문서 통합

**페이지 단위 구분을 제거** 하고 전체 내용을 하나의 연속된 텍스트로 통합합니다. 

#### 통합하는 이유
- **의미적 연속성**: 페이지 경계를 넘나드는 내용을 자연스럽게 연결
- **클러스터링 효과**: 의미적으로 관련된 내용끼리 더 정확한 그룹핑  
- **청킹 최적화**: 내용 기반 분할로 더 의미있는 청크 생성

In [ ]:
# 모든 페이지의 내용을 하나의 연속된 텍스트로 통합
texts = "\n\n".join([doc.page_content for doc in docs])

# 통합된 텍스트의 총 글자 수 확인
len(texts)

### 지능형 텍스트 분할

**RecursiveCharacterTextSplitter** 를 사용하여 통합된 텍스트를 **의미 있는 청크** 로 분할합니다.

#### 분할 매개변수
- **chunk_size**: 500자 - 적절한 의미 단위 보장
- **chunk_overlap**: 100자 - 청크 간 연속성 유지

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정 (500자 청크, 100자 중복)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 통합된 텍스트를 의미 있는 청크들로 분할
split_docs = text_splitter.split_text(texts)

### 분할 결과 확인

**28K글자** 의 대용량 문서가 **79개의 의미있는 청크** 로 분할되었습니다. 이제 이 청크들을 클러스터링하여 대표 문서들을 선별하겠습니다.

In [ ]:
# 분할된 문서 청크의 총 개수 확인
len(split_docs)

### 벡터 임베딩 생성

각 문서 청크를 **의미적 벡터** 로 변환합니다. 여기서는 두 가지 임베딩 모델 옵션을 제시합니다:

1. **Upstage Embeddings**: 한국어에 최적화된 모델
2. **OpenAI Embeddings**: 범용성이 뛰어난 모델

실습에서는 **OpenAI Embeddings** 를 사용하겠습니다.

In [ ]:
from langchain_upstage import UpstageEmbeddings

# Upstage 임베딩 모델 설정 (한국어 최적화)
embeddings = UpstageEmbeddings(model="solar-embedding-1-large-passage")

# 모든 문서 청크를 벡터로 변환
vectors = embeddings.embed_documents(split_docs)

In [ ]:
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 설정 (범용성 우수)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 모든 문서 청크를 벡터로 변환
vectors = embeddings.embed_documents(split_docs)

### KMeans 클러스터링

**79개 문서** 를 **10개 클러스터** 로 그룹핑합니다. 

#### 클러스터 개수 선택 기준
- **너무 적으면**: 다양성 부족, 정보 손실  
- **너무 많으면**: 효율성 감소, 중복 증가
- **10개**: 79개 문서에 대한 **최적 균형점** (약 13% 선별)

In [ ]:
from sklearn.cluster import KMeans

# 클러스터 수 설정 (문서 내용과 복잡도에 따라 조정 가능)
num_clusters = 10

# KMeans 클러스터링 수행 (random_state로 재현 가능한 결과 보장)
kmeans = KMeans(n_clusters=num_clusters, random_state=123).fit(vectors)

### 클러스터링 결과 확인

각 문서가 **어느 클러스터에 속하는지** 라벨링된 결과를 확인합니다. 0-9까지의 숫자가 각 클러스터를 나타냅니다.

In [ ]:
# 각 문서가 할당된 클러스터 번호 확인 (0-9)
kmeans.labels_

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 경고 메시지 제거
import warnings

warnings.filterwarnings("ignore")

# t-SNE를 사용하여 고차원 벡터를 2차원으로 축소 (시각화를 위함)
tsne = TSNE(n_components=2, random_state=42)
reduced_data_tsne = tsne.fit_transform(np.array(vectors))

# seaborn 스타일 설정
sns.set_style("white")

# 클러스터별로 색깔을 다르게 하여 2차원 산점도 그리기
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=reduced_data_tsne[:, 0],  # X축: 첫 번째 차원
    y=reduced_data_tsne[:, 1],  # Y축: 두 번째 차원
    hue=kmeans.labels_,  # 색상: 클러스터 번호에 따라 구분
    palette="deep",  # 색상 팔레트
    s=100,  # 점 크기
)

# 그래프 레이블 및 제목 설정
plt.xlabel("Dimension 1", fontsize=12)
plt.ylabel("Dimension 2", fontsize=12)
plt.title("Clustered Embeddings", fontsize=16)
plt.legend(title="Cluster", title_fontsize=12)

# 배경색 설정
plt.gcf().patch.set_facecolor("white")

plt.tight_layout()
plt.show()

### 클러스터 대표 문서 선별

각 클러스터에서 **중심점에 가장 가까운 문서** 를 찾아 대표 문서로 선택합니다.

#### 선별 원리
- **클러스터 중심점**: 해당 그룹의 평균적 특성을 나타내는 지점
- **최소 거리**: 중심점과 가장 가까운 = 가장 대표적인 문서
- **효율적 샘플링**: 전체 정보를 유지하면서 처리량 대폭 감소

In [ ]:
import numpy as np

# 각 클러스터의 중심점에 가장 가까운 문서를 저장할 빈 리스트 생성
closest_indices = []

# 각 클러스터별로 중심점에 가장 가까운 문서 찾기
for i in range(num_clusters):

    # 현재 클러스터의 중심점과 모든 문서 간의 유클리드 거리 계산
    distances = np.linalg.norm(vectors - kmeans.cluster_centers_[i], axis=1)

    # 거리가 가장 짧은(가장 가까운) 문서의 인덱스 찾기
    closest_index = np.argmin(distances)

    # 해당 인덱스를 대표 문서 리스트에 추가
    closest_indices.append(closest_index)

In [ ]:
# 선별된 대표 문서들의 인덱스 확인
closest_indices

### 순서 정렬

**Map-Refine 방식** 에서는 문서의 순서가 중요하므로, 선택된 인덱스를 **오름차순으로 정렬** 하여 원래 문서의 흐름을 유지합니다.

In [ ]:
# 문서 순서 유지를 위해 인덱스를 오름차순으로 정렬
selected_indices = sorted(closest_indices)

# 정렬된 인덱스 확인
selected_indices

### 선별된 문서 객체 생성

선택된 **10개의 대표 문서** 를 LangChain의 `Document` 객체로 변환합니다. 이는 Map-Refine 체인에서 요구하는 입력 형식입니다.

In [ ]:
from langchain_core.documents import Document

# 선별된 인덱스에 해당하는 문서들을 Document 객체로 변환
selected_docs = [Document(page_content=split_docs[doc]) for doc in selected_indices]

# 생성된 Document 객체들 확인
selected_docs

In [ ]:
# 이전에 정의한 Map-Refine 체인을 사용하여 선별된 10개 문서를 요약
refined_summary = map_refine_chain.invoke(selected_docs)

In [ ]:
# Clustering-Map-Refine 방식의 최종 요약 결과 출력
print(refined_summary)